<a href="https://colab.research.google.com/github/chetna-0608/CODSOFT/blob/main/handwritten_digit_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

title of the project: hand written digit recognition


OBJECTIVE:
Digit recognition using SVM


DATA SOURCE:
captured images from screen

**IMPORT LIBRARY**

In [ ]:
import pyscreenshot as ImageGrab
import time
import cv2
import csv
import glob
import os
import pandas as pd
from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import metrics
import joblib
import numpy as np
import matplotlib.pyplot as plt




**IMPORT DATA**

In [ ]:
images_folder = "C:\\Users\\Hp\\Downloads\\ybi\\captured_img\\"

for i in range(0, 10):
    time.sleep(4)
    im = ImageGrab.grab(bbox=(160, 240, 622, 900))  # x1, y1, x2, y2
    print("saved......", i)
    im.save(images_folder + str(i) + '.png')
    print("clear screen now and redraw now........")

**DESCRIBE DATA**

In [ ]:


# Create header for the CSV file
header = ["label"]
for i in range(0, 784):
    header.append("pixel" + str(i))

# Open the CSV file once and write the header
with open('dataset.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)

# Process images from all subfolders
base_folder = "C:\\Users\\Hp\\Downloads\\ybi\\captured_img\\"

for label in range(10):  # Assuming folders are named 0 to 9
    folder_path = os.path.join(base_folder, str(label))
    dirList = glob.glob(folder_path + "/*.png")

    # Process each image in the folder
    for img_path in dirList:
        im = cv2.imread(img_path)
        if im is None:
            print(f"Error reading image {img_path}")
            continue

        im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
        im_gray = cv2.GaussianBlur(im_gray, (15, 15), 0)
        ret, im_th = cv2.threshold(im_gray, 100, 255, cv2.THRESH_BINARY)
        roi = cv2.resize(im_th, (28, 28), interpolation=cv2.INTER_AREA)

        data = [label]
        rows, cols = roi.shape

        # Fill the data array with pixels one by one
        for i in range(rows):
            for j in range(cols):
                k = roi[i, j]
                if k > 100:
                    k = 1
                else:
                    k = 0
                data.append(k)

        # Append the data row to the CSV file
        with open('dataset.csv', 'a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(data)


**DATA VISUALIZAION**

In [ ]:

import pandas as pd
from sklearn.utils import shuffle
from sklearn.svm import SVC
import joblib
import os

# Read and shuffle the dataset
data = pd.read_csv('dataset.csv')
data = shuffle(data)
print(f"Number of rows in the dataset: {len(data)}")
print(data)

**DATA PREPROCESSING**: Grayscale conversion, Gaussian blur, thresholding, resizing

**Define Target Variable (y) and Feature Variables (X)**

In [ ]:
data = pd.read_csv('dataset.csv')
data = shuffle(data)
X = data.drop(["label"], axis=1)
Y = data["label"]


**TRAIN TEST SPLIT**

In [ ]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=0.2, random_state=42)


**Modeling**

In [ ]:
classifier = SVC(kernel="linear", random_state=6)
classifier.fit(train_x, train_y)
# Create the model directory inside the ybi folder if it doesn't exist
ybi_dir = "C:\\Users\\Hp\\Downloads\\ybi"
model_dir = os.path.join(ybi_dir, "model")
os.makedirs(model_dir, exist_ok=True)

# Save the model
model_path = os.path.join(model_dir, "digit_recognizer")
joblib.dump(classifier, model_path)


**Model Evaluation**

In [ ]:
prediction = classifier.predict(test_x)
accuracy = metrics.accuracy_score(test_y, prediction)
print("Accuracy = ", accuracy)

**Prediction:**
 Using SVM model to predict digits from screen capture

In [ ]:
model_path = "C:/Users/Hp/Downloads/ybi/model/digit_recognizer"
if os.path.exists(model_path):
    model = joblib.load(model_path)
else:
    print(f"Error: Model file '{model_path}' not found.")
    exit()

image_folder = "C:/Users/Hp/img/"
os.makedirs(image_folder, exist_ok=True)

img = ImageGrab.grab(bbox=(60, 170, 400, 500))
img_path = image_folder + "img.png"
img.save(img_path)

im = cv2.imread(img_path)
im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
im_gray = cv2.GaussianBlur(im_gray, (15, 15), 0)
ret, im_th = cv2.threshold(im_gray, 100, 255, cv2.THRESH_BINARY)
roi = cv2.resize(im_th, (28, 28), interpolation=cv2.INTER_AREA)

# Prepare the input data for prediction
X_pred = []
rows, cols = roi.shape
for i in range(rows):
    for j in range(cols):
        k = roi[i, j]
        if k > 100:
            k = 1
        else:
            k = 0
        X_pred.append(k)

X_pred = np.array(X_pred).reshape(1, -1)
predictions = model.predict(X_pred)
print("Prediction:", predictions[0])



**EXPLANATION:**

The model predicts digits based on pixel data extracted from screenshots. The SVM classifier is trained on preprocessed image data and evaluated for accuracy. Captured screenshots are processed to extract features, predict digits, and display results.